# 필요한 library

In [1]:
!pip install konlpy
!pip install koreanize-matplotlib
!pip install transformers
!pip install reportlab
!pip install hangul-romanize

!wget https://fonts.google.com/download?family=Noto%20Sans%20KR -O notosanskr.zip
!unzip notosanskr.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.2 MB/s eta 0:00:00
--2023-10-11 13:10:30--  https://fonts.google.com/download?family=Noto%20Sans%20KR
Resolving fonts.google.com (fonts.google.com)... 64.233.188.139, 64.233.188.113, 64.233.188.100, ...
Connecting to fonts.google.com (fonts.google.com)|64.233.188.139|:443... connected.
HTTP request 

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import re
from konlpy.tag import Okt
from collections import Counter
import konlpy
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import koreanize_matplotlib
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from konlpy.tag import Kkma
import torch
from collections import Counter
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
import os
import copy
from PIL import Image
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from hangul_romanize import Transliter
from hangul_romanize.rule import academic
import unicodedata
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import Paragraph
from reportlab.lib.units import inch


# Noto Sans KR 폰트 등록
pdfmetrics.registerFont(TTFont('NotoSansKR', 'static/NotoSansKR-Regular.ttf'))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/SNU_JYS/NLP/JuheeFile/'
path_graph = '/content/drive/MyDrive/SNU_JYS/NLP/report_try/graphs/'
path_report_try = '/content/drive/My Drive/SNU_JYS/NLP/report_try'

# 실시간 특정 기업 증권 뉴스 10개 크롤링
- OUTPUT : 10개 기업명 리스트 / 10개 종목코드 리스트 / 10개 뉴스 데이터프레임

In [5]:
### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
def ex_tag(sid, sid2, page):
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}&sid2={sid2}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")

    ## 2.
    tag_set = set()  # set 자료형을 사용하여 중복 제거
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_set.add(a["href"])  # set에 추가

    return list(tag_set)  # set을 list로 변환하여 반환

In [6]:
def art_crawl(links, index, specified_date=None):
    art_dic = {}

    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"

    url = links[index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")

    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)

    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)

    # 지정된 날짜와 크롤링된 날짜가 일치하지 않으면 빈 딕셔너리 반환
    extracted_date = date_str.split(". ")[0]
    if specified_date and specified_date != extracted_date:
        return {}

    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)

    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str

    return art_dic

In [7]:
# 조사 리스트 정리
Josa = open(path + 'JosaEomi/JOSA.TXT', "r", encoding='CP949')
Josa_lines = Josa.readlines()
Josa_list = []
for line in Josa_lines:
  Josa_list.append(str(line)[:-1])
Josa_list = Josa_list[:-1]

# 어미 리스트 정리
Eomi = open(path + 'JosaEomi/EOMI.TXT', 'r', encoding='CP949')
Eomi_lines = Eomi.readlines()
Eomi_list = []
for line in Eomi_lines:
  Eomi_list.append(str(line)[:-1])
Eomi_list = Eomi_list[:-1]

# 상장기업 리스트 제작
kospi_com = pd.read_csv(path + 'KOSPI_업종분류현황(0926)_20231004.csv', encoding='CP949')
#kosdaq_com = pd.read_csv(path + 'KOSDAQ_업종분류현황(0926)_20231004.csv', encoding='CP949')
#company_list = list(kospi_com['종목명']) + list(kosdaq_com['종목명'])
company_list = list(kospi_com['종목명'])   ### 코스피 회사들만

# 업종별 상장기업 목록 제작
business = list(kospi_com['업종명'].unique())
business_dict = {}
for i in range(len(business)):
  business_dict[business[i]] = []
for i in range(len(kospi_com)):
  business_dict[kospi_com.loc[i]['업종명']].append(kospi_com.loc[i]['종목명'])

In [8]:
# 조사나 어미를 제거하는 함수
def remove_josa_eomi(original_word, josa_list, eomi_list):
    word = original_word

    # 조사 제거
    for josa in sorted(josa_list, key=len, reverse=True):  # 긴 조사부터 확인
        if word.endswith(josa):
            word = word[:-len(josa)]

    # 어미 제거
    for eomi in sorted(eomi_list, key=len, reverse=True):  # 긴 어미부터 확인
        if word.endswith(eomi):
            word = word[:-len(eomi)]

    # 조사나 어미가 없는 경우
    return word

In [9]:
# 명사를 추출하는 함수 : input = 뉴스기사 1개의 딕셔너리
def extract_noun(example):
  # []나 () 안에 있는 문자열을 제거
  example = re.sub(r'\[.*?\]', '', example)
  example = re.sub(r'\(.*?\)', '', example)
  # '와 " 와 . 와 = 와 , 문자 자체만 제거
  example = example.replace('"', '').replace("'", "").replace("=","").replace(",","")
  # 마침표(.) 또는 공백 문자(\s)를 기준으로 문자열을 분리
  example = re.split(r'[.\s]', example)
  # 숫자가 있으면 단어 자체를 삭제
  example = [word for word in example if not any(char.isdigit() for char in word)]

  # 각 단어에 대해 조사나 어미 제거
  result = [remove_josa_eomi(word, Josa_list, Eomi_list) for word in example]

  return(result)

In [10]:
# 기업과 종목코드 match 딕셔너리 만들기 : {기업 : 종목코드}
company_code = {}

for i in range(len(kospi_com[['종목코드', '종목명']])):
  company_code[str(kospi_com.loc[i]['종목명'])] = str(kospi_com.loc[i]['종목코드'])
#for i in range(len(kosdaq_com[['종목코드', '종목명']])):
 # company_code[str(kosdaq_com.loc[i]['종목명'])] = str(kosdaq_com.loc[i]['종목코드'])

In [11]:
# 뉴스 안 명사 리스트에 들어있는 특정 기업 1개 이름&코드 추출 + 1개 기업 특정 뉴스만 모으는 함수
def find_company(result, i, company_in_news, code_in_news, selected_news):
  matching_company = set([noun for noun in result if noun in company_list])

  if (len(matching_company)) == 1:
    # 기업명(종목명)
    company = matching_company.pop()
    if company in company_in_news:
      return(company_in_news, code_in_news, select_news)
    elif company == '대상':
      return(company_in_news, code_in_news, select_news)
    else:
      company_in_news.append(company)
      # 종목코드
      code = company_code[company]
      code_in_news.append(code)
      # 해당 기업이 들어간 뉴스
      select_news.append(art_df.iloc[i])  # i 번째 행을 선택
  return(company_in_news, code_in_news, select_news)

In [12]:
### 실행
from datetime import datetime
news_num =0 ; pgn = 0
company_in_news = [] ; code_in_news = [] ; select_news = []

while news_num < 10:
  pgn += 1
  # 경제섹션(101), 증권파트(258), n페이지의 링크 가져오기
  links = ex_tag(101,258,pgn)

  # 링크 안에 있는 뉴스들 크롤링 -> 데이터프레임화
  artdic_lst = []
  today = datetime.today()
  formatted_date = today.strftime("%Y.%m.%d")
  for i in tqdm(range(len(links))):
    art_dic = art_crawl(links, i, specified_date=formatted_date)  # 오늘 날짜에 해당하는 뉴스만 가져오기
    if art_dic != {}:
        art_dic["url"] = links[i]
        artdic_lst.append(art_dic)
  art_df = pd.DataFrame(artdic_lst)

  # 데이터프레임에 있는 각 뉴스들마다 반복해서 : 명사추출 -> 특정기업뉴스에 대한 기업명&코드&뉴스전문
  for i in range(len(art_df)):
    example = art_df['main'][i]
    result = extract_noun(example)
    company_in_news, code_in_news, select_news = find_company(result ,i, company_in_news, code_in_news, select_news)
    news_num = len(select_news)
    if news_num == 10 : break

final_company_list = company_in_news
final_code_list = code_in_news
final_news_list = (pd.DataFrame(select_news)).reset_index(drop=True)

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [13]:
final_news_list

,title,date,main,url
0,"역대 최대 실적…북미 강자 LG엔솔, 3분기 영업익 40% ‘껑충’",2023.10.11. 오후 3:52,LG에너지솔루션 미국 미시간주 전기차 배터리 공장. [사진 출처 = 연합뉴스]LG에...,https://n.news.naver.com/mnews/article/009/000...
1,"“땡큐, 현대로템” 우크라戰 피란민 ‘생명줄’ 된 韓 철도차량",2023.10.11. 오전 9:06,현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는 데 핵...,https://n.news.naver.com/mnews/article/366/000...
2,"""반도체 초기술은 '뉴진스 ETA'처럼""…곽노정 사장이 말한 SK하이닉스 비전은",2023.10.11. 오후 6:14,카이스트 찾아 특별 강연메모리 반도체 활용 범위 확대 추세핵심 경쟁력으로 ‘초기술’...,https://n.news.naver.com/mnews/article/011/000...
3,"'고용 세습' 건들지마…기아 노조, 임단협 결렬 선언 ""내일부터 파업""",2023.10.11. 오전 8:43,"12~13일, 17~19일 8시간씩 파업교섭일엔 정상 근무…추가 교섭 여지\n\n\...",https://n.news.naver.com/mnews/article/421/000...
4,"'어려운 소상공인 적극 지원' …경기신보, '경기 기회UP 특례보증' 시행",2023.10.11. 오전 8:55,경영애로 겪고 있거나 고금리대출 7% 보유한 소상공인에 저금리 장기대출대출금리 이차...,https://n.news.naver.com/mnews/article/119/000...
5,"""MZ들 푹 빠지겠네""…올 연말 '스타필드 수원' 베일 벗는다 [송영찬의 신통유통]",2023.10.11. 오후 3:14,오는 12월 경기 수원시 정자동에 개점 예정인 스타필드 수원의 조감도./ 신세계프라...,https://n.news.naver.com/mnews/article/015/000...
6,"광주시의회 ""대유 협력사 줄도산 위기…피해 대책 마련해야""",2023.10.11. 오후 5:29,광주시의회 본회의장. (사진=광주시의회 제공) *재판매 및 DB 금지[광주=뉴시스]...,https://n.news.naver.com/mnews/article/003/001...
7,"이종호 ""R&D 구조개혁 시기…낡은 관행·비효율 걷어내야""",2023.10.11. 오전 11:06,[이종호 과학기술정보통신부 장관. (사진=연합뉴스)]이종호 과학기술정보통신부 장관이...,https://n.news.naver.com/mnews/article/374/000...
8,공매도 제도 손본다…2차전지 불법공매도 정조준,2023.10.11. 오전 5:31,"정무위 강훈식 의원, 자본시장법 개정 추진차입·담보 차이 해소, 전산시스템 도입 규...",https://n.news.naver.com/mnews/article/018/000...
9,"KB금융, 23개 스타트업 'KB스타터스' 선정 ""유니콘 되게 지원""",2023.10.11. 오후 6:16,[이 기사에 나온 스타트업에 대한 보다 다양한 기업정보는 유니콘팩토리 빅데이터 플랫...,https://n.news.naver.com/mnews/article/008/000...


In [14]:
# DataFrame 생성
df_links = pd.DataFrame(final_news_list, columns=['url'])

# 출력
print(df_links)

                                                 url
0  https://n.news.naver.com/mnews/article/009/000...
1  https://n.news.naver.com/mnews/article/366/000...
2  https://n.news.naver.com/mnews/article/011/000...
3  https://n.news.naver.com/mnews/article/421/000...
4  https://n.news.naver.com/mnews/article/119/000...
5  https://n.news.naver.com/mnews/article/015/000...
6  https://n.news.naver.com/mnews/article/003/001...
7  https://n.news.naver.com/mnews/article/374/000...
8  https://n.news.naver.com/mnews/article/018/000...
9  https://n.news.naver.com/mnews/article/008/000...


In [15]:
final_news_list

,title,date,main,url
0,"역대 최대 실적…북미 강자 LG엔솔, 3분기 영업익 40% ‘껑충’",2023.10.11. 오후 3:52,LG에너지솔루션 미국 미시간주 전기차 배터리 공장. [사진 출처 = 연합뉴스]LG에...,https://n.news.naver.com/mnews/article/009/000...
1,"“땡큐, 현대로템” 우크라戰 피란민 ‘생명줄’ 된 韓 철도차량",2023.10.11. 오전 9:06,현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는 데 핵...,https://n.news.naver.com/mnews/article/366/000...
2,"""반도체 초기술은 '뉴진스 ETA'처럼""…곽노정 사장이 말한 SK하이닉스 비전은",2023.10.11. 오후 6:14,카이스트 찾아 특별 강연메모리 반도체 활용 범위 확대 추세핵심 경쟁력으로 ‘초기술’...,https://n.news.naver.com/mnews/article/011/000...
3,"'고용 세습' 건들지마…기아 노조, 임단협 결렬 선언 ""내일부터 파업""",2023.10.11. 오전 8:43,"12~13일, 17~19일 8시간씩 파업교섭일엔 정상 근무…추가 교섭 여지\n\n\...",https://n.news.naver.com/mnews/article/421/000...
4,"'어려운 소상공인 적극 지원' …경기신보, '경기 기회UP 특례보증' 시행",2023.10.11. 오전 8:55,경영애로 겪고 있거나 고금리대출 7% 보유한 소상공인에 저금리 장기대출대출금리 이차...,https://n.news.naver.com/mnews/article/119/000...
5,"""MZ들 푹 빠지겠네""…올 연말 '스타필드 수원' 베일 벗는다 [송영찬의 신통유통]",2023.10.11. 오후 3:14,오는 12월 경기 수원시 정자동에 개점 예정인 스타필드 수원의 조감도./ 신세계프라...,https://n.news.naver.com/mnews/article/015/000...
6,"광주시의회 ""대유 협력사 줄도산 위기…피해 대책 마련해야""",2023.10.11. 오후 5:29,광주시의회 본회의장. (사진=광주시의회 제공) *재판매 및 DB 금지[광주=뉴시스]...,https://n.news.naver.com/mnews/article/003/001...
7,"이종호 ""R&D 구조개혁 시기…낡은 관행·비효율 걷어내야""",2023.10.11. 오전 11:06,[이종호 과학기술정보통신부 장관. (사진=연합뉴스)]이종호 과학기술정보통신부 장관이...,https://n.news.naver.com/mnews/article/374/000...
8,공매도 제도 손본다…2차전지 불법공매도 정조준,2023.10.11. 오전 5:31,"정무위 강훈식 의원, 자본시장법 개정 추진차입·담보 차이 해소, 전산시스템 도입 규...",https://n.news.naver.com/mnews/article/018/000...
9,"KB금융, 23개 스타트업 'KB스타터스' 선정 ""유니콘 되게 지원""",2023.10.11. 오후 6:16,[이 기사에 나온 스타트업에 대한 보다 다양한 기업정보는 유니콘팩토리 빅데이터 플랫...,https://n.news.naver.com/mnews/article/008/000...


In [16]:
# 기업명 df 저장
final_company_df = pd.DataFrame(final_company_list, columns=['Company Name'])

In [17]:
final_news_list

,title,date,main,url
0,"역대 최대 실적…북미 강자 LG엔솔, 3분기 영업익 40% ‘껑충’",2023.10.11. 오후 3:52,LG에너지솔루션 미국 미시간주 전기차 배터리 공장. [사진 출처 = 연합뉴스]LG에...,https://n.news.naver.com/mnews/article/009/000...
1,"“땡큐, 현대로템” 우크라戰 피란민 ‘생명줄’ 된 韓 철도차량",2023.10.11. 오전 9:06,현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는 데 핵...,https://n.news.naver.com/mnews/article/366/000...
2,"""반도체 초기술은 '뉴진스 ETA'처럼""…곽노정 사장이 말한 SK하이닉스 비전은",2023.10.11. 오후 6:14,카이스트 찾아 특별 강연메모리 반도체 활용 범위 확대 추세핵심 경쟁력으로 ‘초기술’...,https://n.news.naver.com/mnews/article/011/000...
3,"'고용 세습' 건들지마…기아 노조, 임단협 결렬 선언 ""내일부터 파업""",2023.10.11. 오전 8:43,"12~13일, 17~19일 8시간씩 파업교섭일엔 정상 근무…추가 교섭 여지\n\n\...",https://n.news.naver.com/mnews/article/421/000...
4,"'어려운 소상공인 적극 지원' …경기신보, '경기 기회UP 특례보증' 시행",2023.10.11. 오전 8:55,경영애로 겪고 있거나 고금리대출 7% 보유한 소상공인에 저금리 장기대출대출금리 이차...,https://n.news.naver.com/mnews/article/119/000...
5,"""MZ들 푹 빠지겠네""…올 연말 '스타필드 수원' 베일 벗는다 [송영찬의 신통유통]",2023.10.11. 오후 3:14,오는 12월 경기 수원시 정자동에 개점 예정인 스타필드 수원의 조감도./ 신세계프라...,https://n.news.naver.com/mnews/article/015/000...
6,"광주시의회 ""대유 협력사 줄도산 위기…피해 대책 마련해야""",2023.10.11. 오후 5:29,광주시의회 본회의장. (사진=광주시의회 제공) *재판매 및 DB 금지[광주=뉴시스]...,https://n.news.naver.com/mnews/article/003/001...
7,"이종호 ""R&D 구조개혁 시기…낡은 관행·비효율 걷어내야""",2023.10.11. 오전 11:06,[이종호 과학기술정보통신부 장관. (사진=연합뉴스)]이종호 과학기술정보통신부 장관이...,https://n.news.naver.com/mnews/article/374/000...
8,공매도 제도 손본다…2차전지 불법공매도 정조준,2023.10.11. 오전 5:31,"정무위 강훈식 의원, 자본시장법 개정 추진차입·담보 차이 해소, 전산시스템 도입 규...",https://n.news.naver.com/mnews/article/018/000...
9,"KB금융, 23개 스타트업 'KB스타터스' 선정 ""유니콘 되게 지원""",2023.10.11. 오후 6:16,[이 기사에 나온 스타트업에 대한 보다 다양한 기업정보는 유니콘팩토리 빅데이터 플랫...,https://n.news.naver.com/mnews/article/008/000...


# KoBERT 요약

In [18]:
# 모델과 문장 분리를 위한 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [19]:
def summarization(df, tokenizer, model):

    # Ensure 'summary' column exists in the DataFrame
    if 'summary' not in df.columns:
        df['summary'] = ''

    for i in range(len(df)):
        try:
            news_text = df.loc[i, 'main']  # 요약 원문

            # 토크나이저를 사용하여 뉴스기사 원문을 모델이 인식할 수 있는 토큰 형태로 바꿔줌
            input_ids = tokenizer.encode(news_text, return_tensors="pt")

            # Ensure input length is within model's acceptable range
            if len(input_ids[0]) > model.config.max_position_embeddings:
                return "Input is too long for the model to handle."

            # 숫자로 변환된 뉴스 기사를 모델의 input으로 사용하여 요약문을 생성
            summary_text_ids = model.generate(
                input_ids=input_ids,
                bos_token_id=model.config.bos_token_id,
                eos_token_id=model.config.eos_token_id,
                length_penalty=1.0,   # 길이에 대한 penalty, 1보다 작은 경우 더 짧은 문장, 클 경우 더 긴 문장을 생성하도록 유도
                max_length=100,       # 요약문의 최대 길이 설정
                min_length=32,        # 요약문의 최소 길이 설정
                num_beams=4           # 문장 생성 시 다음 단어를 탐색하는 영역의 개수
            )

            df.loc[i, 'summary'] = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

        except Exception as e:
            df.loc[i, 'summary'] = f"An error occurred: {e}"

    return df

In [20]:
summarization(final_news_list, tokenizer, model)

for i in range(len(final_news_list)) :
  print(i, ":" , final_news_list.loc[i,'summary'])

0 : LG에너지솔루션의 3분기 실적은 미국 인플레이션감축법에 따른 첨단제조생산세액공제로 인한 영향이 컸으며, 올해 말부터 2025년까지 미국 미시간 공장에 총 4조원을 투자하여 토요타 전용 배터리 셀·모듈 생산라인을 구축할 계획이다.
1 : 11일 현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는 데 핵심 역할을 수행한 공로로 현지 정부로부터 감사의 의미를 담은 팸플릿을 전달받았는데 팸플릿 오른쪽 상단에는 현대로템의 철도차량은 지난해 봄 자국 아이들과 어머니들을 위한 ’생명줄’중 하나가 됐다며 앞으로도 계속 그들(자국민)을 부탁한다는 문구가 적힌 우크라이나 재건부(Minist
2 : 곽노정 SK하이닉스 사장은 11일 대전 유성구 한국과학기술원(KAIST)에서 ‘초기술로 세상을 더 행복하게’라는 주제로 특별강연을 진행한 가운데 곽 사장은 ‘초기술’이 기반이 된 ‘굿 메모리(Good Memory)’를 지속해서 만들어낼 것이며 반도체 미래 인재들이 SK하이닉스에 모일 수 있도록 최선의 노력을 다할 것이라고 말했다.
3 : 기아차 노동조합이 올해 임금 및 단체협상(임단협)에서 쟁점이 됐던 '장기근속자 자녀 우선 채용'에서 합의점을 찾지 못하여 최종 결렬을 선언하였고 '장기근속자 자녀 우선 채용'에서 합의점을 찾지 못하여 잠정 파업에 들어가지만, 교섭이 있는 날에는 정상근무를 진행한다고 밝혔다.
4 : 경기신용보증재단(이하 경기신보)은 금융비용 상승으로 인한 자금애로 해소를 최우선으로 판단, 대출기간동안 은행이 정한 대출금리에서 2%포인트 이자를 지원하는 '경기 기회UP 특례보증(이하 기회UP 특례보증)'을 시행하여 경제위기 연착륙을 김동연 경기도지사가 적극 재정 추진으로 시행하게 됐다.
5 : 1세계프라퍼티는 지체됐던 지역 상인들과의 상생 협약이 모두 완료된 덕에 MZ세대(밀레니얼+Z세대)를 정조준해 구성한 ‘스타필드 수원’을 오는 12월 경기 수원시 정자동에 개점한다고 11일 밝혔다.
6 : 광주시의회는 11일 대내외 악재로 수출마저 감

# 감정분석


*   pretrain 모델 불러오기



In [21]:
# 토크나이저와 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/SNU_JYS/NLP/trained_model')  # 여기에 저장된 모델의 경로를 입력합니다.

# 모델을 평가 모드로 전환
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12


*   test



In [22]:
art_df = pd.DataFrame(final_news_list)
art_df

,title,date,main,url,summary
0,"역대 최대 실적…북미 강자 LG엔솔, 3분기 영업익 40% ‘껑충’",2023.10.11. 오후 3:52,LG에너지솔루션 미국 미시간주 전기차 배터리 공장. [사진 출처 = 연합뉴스]LG에...,https://n.news.naver.com/mnews/article/009/000...,LG에너지솔루션의 3분기 실적은 미국 인플레이션감축법에 따른 첨단제조생산세액공제로 ...
1,"“땡큐, 현대로템” 우크라戰 피란민 ‘생명줄’ 된 韓 철도차량",2023.10.11. 오전 9:06,현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는 데 핵...,https://n.news.naver.com/mnews/article/366/000...,11일 현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는...
2,"""반도체 초기술은 '뉴진스 ETA'처럼""…곽노정 사장이 말한 SK하이닉스 비전은",2023.10.11. 오후 6:14,카이스트 찾아 특별 강연메모리 반도체 활용 범위 확대 추세핵심 경쟁력으로 ‘초기술’...,https://n.news.naver.com/mnews/article/011/000...,곽노정 SK하이닉스 사장은 11일 대전 유성구 한국과학기술원(KAIST)에서 ‘초기...
3,"'고용 세습' 건들지마…기아 노조, 임단협 결렬 선언 ""내일부터 파업""",2023.10.11. 오전 8:43,"12~13일, 17~19일 8시간씩 파업교섭일엔 정상 근무…추가 교섭 여지\n\n\...",https://n.news.naver.com/mnews/article/421/000...,기아차 노동조합이 올해 임금 및 단체협상(임단협)에서 쟁점이 됐던 '장기근속자 자녀...
4,"'어려운 소상공인 적극 지원' …경기신보, '경기 기회UP 특례보증' 시행",2023.10.11. 오전 8:55,경영애로 겪고 있거나 고금리대출 7% 보유한 소상공인에 저금리 장기대출대출금리 이차...,https://n.news.naver.com/mnews/article/119/000...,경기신용보증재단(이하 경기신보)은 금융비용 상승으로 인한 자금애로 해소를 최우선으로...
5,"""MZ들 푹 빠지겠네""…올 연말 '스타필드 수원' 베일 벗는다 [송영찬의 신통유통]",2023.10.11. 오후 3:14,오는 12월 경기 수원시 정자동에 개점 예정인 스타필드 수원의 조감도./ 신세계프라...,https://n.news.naver.com/mnews/article/015/000...,1세계프라퍼티는 지체됐던 지역 상인들과의 상생 협약이 모두 완료된 덕에 MZ세대(밀...
6,"광주시의회 ""대유 협력사 줄도산 위기…피해 대책 마련해야""",2023.10.11. 오후 5:29,광주시의회 본회의장. (사진=광주시의회 제공) *재판매 및 DB 금지[광주=뉴시스]...,https://n.news.naver.com/mnews/article/003/001...,광주시의회는 11일 대내외 악재로 수출마저 감소돼 위기가 심화되는 와중에 (주)위...
7,"이종호 ""R&D 구조개혁 시기…낡은 관행·비효율 걷어내야""",2023.10.11. 오전 11:06,[이종호 과학기술정보통신부 장관. (사진=연합뉴스)]이종호 과학기술정보통신부 장관이...,https://n.news.naver.com/mnews/article/374/000...,"이종호 과학기술정보통신부 장관은 국정감사에서 ""낡은 관행과 비효율을 걷어내고 최고 ..."
8,공매도 제도 손본다…2차전지 불법공매도 정조준,2023.10.11. 오전 5:31,"정무위 강훈식 의원, 자본시장법 개정 추진차입·담보 차이 해소, 전산시스템 도입 규...",https://n.news.naver.com/mnews/article/018/000...,올해 불법 공매도 제재 건수가 역대 최다 규모를 기록한 가운데 ‘기울어진 운동장’ ...
9,"KB금융, 23개 스타트업 'KB스타터스' 선정 ""유니콘 되게 지원""",2023.10.11. 오후 6:16,[이 기사에 나온 스타트업에 대한 보다 다양한 기업정보는 유니콘팩토리 빅데이터 플랫...,https://n.news.naver.com/mnews/article/008/000...,KB금융 그룹은 11일 우수한 기술을 갖고도 자금 부족이나 인력 등에 어려움을 겪는...


In [23]:
kkma = Kkma()

# 각 뉴스 기사를 문장별로 나누기
art_df['sentences'] = art_df['main'].apply(kkma.sentences)

# 각 문장의 감정 분석
def get_sentiment(model, tokenizer, sentence):
    model.eval()  # 모델을 평가 모드로 전환
    with torch.no_grad():  # 그레디언트 계산 비활성화
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=128)
        outputs = model(**inputs)
        sentiment = torch.argmax(outputs.logits, dim=-1).item()
    return sentiment

# 데이터프레임을 문장 단위로 나누고 감정 분석 결과를 라벨에 넣기
rows = []
for index, row in art_df.iterrows():
    for sentence in row['sentences']:
        sentiment = get_sentiment(model, tokenizer, sentence)
        rows.append({'sentence': sentence, 'sentiment_label': sentiment})

# 새로운 데이터프레임 생성
sentence_df = pd.DataFrame(rows)

# 출력
print(sentence_df)

                                              sentence  sentiment_label
0    LG 에너지 솔루션 미국 미시간 주 전 기차 배터리 공장. [ 사진 출처 = 연합 ...                1
1                     북미 지역 전 기차 시장이 성장한 영향으로 풀이 된다.LG                1
2                          엔솔은 11일 올 3 분기 잠정 실적을 발표했다.                1
3    LG 엔솔의 3 분기 매출액은 8조 2235억원으로 지난해 같은 기간보다 7.5% ...                1
4                  영업이익은 같은 기간 40.1% 증가한 7312억원을 기록했다.                1
..                                                 ...              ...
156  해마다 상· 하반기로 나눠 혁신적인 스타트 업들을 선발하고 성장 단계별 다양한 지원...                0
157  앞서 픤 디, 라이 언 로켓, 키 뮤 등 스타트 업들이 KB 스타터 스에 선발된 바...                0
158  KB 금융 관계자는 "2023 년 하반기 KB 스타터 스에는 AI 기술을 접목해 비...                1
159  그러면서 " 이 스타트 업들이 차세대 유니콘 기업으로 성장할 수 있도록 지원함은 물...                1
160                 [ 머니 투데이 스타트 업 미디어 플랫폼 ' 유니콘 팩토리']                0

[161 rows x 2 columns]


In [24]:
kkma = Kkma()

# 각 뉴스 기사를 문장별로 나누기
art_df['sentences'] = art_df['main'].apply(kkma.sentences)

def get_sentiment(model, tokenizer, sentence):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=128)
        outputs = model(**inputs)
        sentiment = torch.argmax(outputs.logits, dim=-1).item()
    return sentiment

# 각 뉴스의 문장들을 감정 분석한 후, 전체적인 뉴스의 감정을 결정
news_sentiments = []
for index, row in art_df.iterrows():
    sentiments = [get_sentiment(model, tokenizer, sentence) for sentence in row['sentences']]
    sentiment_count = Counter(sentiments)
    dominant_sentiment = sentiment_count.most_common(1)[0][0]
    news_sentiments.append(dominant_sentiment)

art_df['dominant_sentiment'] = news_sentiments

# 출력
print(art_df[['main', 'dominant_sentiment']])

                                                main  dominant_sentiment
0  LG에너지솔루션 미국 미시간주 전기차 배터리 공장. [사진 출처 = 연합뉴스]LG에...                   1
1  현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는 데 핵...                   1
2  카이스트 찾아 특별 강연메모리 반도체 활용 범위 확대 추세핵심 경쟁력으로 ‘초기술’...                   1
3  12~13일, 17~19일 8시간씩 파업교섭일엔 정상 근무…추가 교섭 여지\n\n\...                   0
4  경영애로 겪고 있거나 고금리대출 7% 보유한 소상공인에 저금리 장기대출대출금리 이차...                   2
5  오는 12월 경기 수원시 정자동에 개점 예정인 스타필드 수원의 조감도./ 신세계프라...                   0
6  광주시의회 본회의장. (사진=광주시의회 제공) *재판매 및 DB 금지[광주=뉴시스]...                   2
7  [이종호 과학기술정보통신부 장관. (사진=연합뉴스)]이종호 과학기술정보통신부 장관이...                   2
8  정무위 강훈식 의원, 자본시장법 개정 추진차입·담보 차이 해소, 전산시스템 도입 규...                   2
9  [이 기사에 나온 스타트업에 대한 보다 다양한 기업정보는 유니콘팩토리 빅데이터 플랫...                   0


In [25]:
final = pd.DataFrame(art_df)
final

,title,date,main,url,summary,sentences,dominant_sentiment
0,"역대 최대 실적…북미 강자 LG엔솔, 3분기 영업익 40% ‘껑충’",2023.10.11. 오후 3:52,LG에너지솔루션 미국 미시간주 전기차 배터리 공장. [사진 출처 = 연합뉴스]LG에...,https://n.news.naver.com/mnews/article/009/000...,LG에너지솔루션의 3분기 실적은 미국 인플레이션감축법에 따른 첨단제조생산세액공제로 ...,[LG 에너지 솔루션 미국 미시간 주 전 기차 배터리 공장. [ 사진 출처 = 연합...,1
1,"“땡큐, 현대로템” 우크라戰 피란민 ‘생명줄’ 된 韓 철도차량",2023.10.11. 오전 9:06,현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는 데 핵...,https://n.news.naver.com/mnews/article/366/000...,11일 현대로템은 우크라이나에 납품한 철도차량이 전쟁 피란민들을 안전하게 대비시키는...,[현대 로템은 우 크라 이 나에 납품한 철도차량이 전쟁 피란 민들을 안전하게 대비시...,1
2,"""반도체 초기술은 '뉴진스 ETA'처럼""…곽노정 사장이 말한 SK하이닉스 비전은",2023.10.11. 오후 6:14,카이스트 찾아 특별 강연메모리 반도체 활용 범위 확대 추세핵심 경쟁력으로 ‘초기술’...,https://n.news.naver.com/mnews/article/011/000...,곽노정 SK하이닉스 사장은 11일 대전 유성구 한국과학기술원(KAIST)에서 ‘초기...,[카 이스트 찾아 특별 강연 메모리 반도체 활용 범위 확대 추세 핵심 경쟁력으로 ‘...,1
3,"'고용 세습' 건들지마…기아 노조, 임단협 결렬 선언 ""내일부터 파업""",2023.10.11. 오전 8:43,"12~13일, 17~19일 8시간씩 파업교섭일엔 정상 근무…추가 교섭 여지\n\n\...",https://n.news.naver.com/mnews/article/421/000...,기아차 노동조합이 올해 임금 및 단체협상(임단협)에서 쟁점이 됐던 '장기근속자 자녀...,"[12~13 일, 17~19 일 8 시간씩 파업 교섭 일엔 정상 근무… 추가 교섭 ...",0
4,"'어려운 소상공인 적극 지원' …경기신보, '경기 기회UP 특례보증' 시행",2023.10.11. 오전 8:55,경영애로 겪고 있거나 고금리대출 7% 보유한 소상공인에 저금리 장기대출대출금리 이차...,https://n.news.naver.com/mnews/article/119/000...,경기신용보증재단(이하 경기신보)은 금융비용 상승으로 인한 자금애로 해소를 최우선으로...,[경영 애로 겪고 있거나 고금리대출 7% 보 유한 소 상공인에 저금리 장기대출대출금...,2
5,"""MZ들 푹 빠지겠네""…올 연말 '스타필드 수원' 베일 벗는다 [송영찬의 신통유통]",2023.10.11. 오후 3:14,오는 12월 경기 수원시 정자동에 개점 예정인 스타필드 수원의 조감도./ 신세계프라...,https://n.news.naver.com/mnews/article/015/000...,1세계프라퍼티는 지체됐던 지역 상인들과의 상생 협약이 모두 완료된 덕에 MZ세대(밀...,[오는 12월 경기 수원시 정자동에 개점 예정인 스타 필드 수원의 조감도./ 신세계...,0
6,"광주시의회 ""대유 협력사 줄도산 위기…피해 대책 마련해야""",2023.10.11. 오후 5:29,광주시의회 본회의장. (사진=광주시의회 제공) *재판매 및 DB 금지[광주=뉴시스]...,https://n.news.naver.com/mnews/article/003/001...,광주시의회는 11일 대내외 악재로 수출마저 감소돼 위기가 심화되는 와중에 (주)위...,[광주시의회 본회의장. ( 사진= 광주시의회 제공) * 재판매 및 DB 금지[ 광주...,2
7,"이종호 ""R&D 구조개혁 시기…낡은 관행·비효율 걷어내야""",2023.10.11. 오전 11:06,[이종호 과학기술정보통신부 장관. (사진=연합뉴스)]이종호 과학기술정보통신부 장관이...,https://n.news.naver.com/mnews/article/374/000...,"이종호 과학기술정보통신부 장관은 국정감사에서 ""낡은 관행과 비효율을 걷어내고 최고 ...",[[ 이종 호 과학기술정보통신 부 장관. ( 사진= 연합 뉴스)] 이종 호 과학기술...,2
8,공매도 제도 손본다…2차전지 불법공매도 정조준,2023.10.11. 오전 5:31,"정무위 강훈식 의원, 자본시장법 개정 추진차입·담보 차이 해소, 전산시스템 도입 규...",https://n.news.naver.com/mnews/article/018/000...,올해 불법 공매도 제재 건수가 역대 최다 규모를 기록한 가운데 ‘기울어진 운동장’ ...,"[정무 위 강 훈 식 의원, 자본 시장법 개정 추진 차입· 담보 차이 해소, 전산시...",2
9,"KB금융, 23개 스타트업 'KB스타터스' 선정 ""유니콘 되게 지원""",2023.10.11. 오후 6:16,[이 기사에 나온 스타트업에 대한 보다 다양한 기업정보는 유니콘팩토리 빅데이터 플랫...,https://n.news.naver.com/mnews/article/008/000...,KB금융 그룹은 11일 우수한 기술을 갖고도 자금 부족이나 인력 등에 어려움을 겪는...,[[ 이 기사에 나온 스타트 업에 대한 보다 다양한 기업정보는 유니콘 팩토리 빅 데...,0


# 실시간 크롤링한 10개 뉴스의 기업명&기업코드로 그래프 그리기
- 3년동안 주가 변화
- 중기(2달) 이동평균선
- 동일 산업 타기업들 주가 평균 변화

In [ ]:
# 주가 데이터 불러오는 함수
def load_stock(code):
  # 뉴스에서 나오는 기업의 종목코드 ticker symbol
  ticker_symbol = code + '.KS'
  # 오늘 날짜 가져오기
  today = datetime.today()

  # 오늘 날짜로부터 3년 2개월 전의 날짜 계산(2달 평균선을 그리기 위함)
  three_years_two_months_ago = today - relativedelta(years=3, months=2)
  # 오늘 날짜로부터 3년 전의 날짜 계산
  three_years_ago = today - relativedelta(years=3)

  # 날짜를 원하는 형식의 문자열로 변환
  end_date = today.strftime('%Y-%m-%d')
  start_date_for_cal = three_years_two_months_ago.strftime('%Y-%m-%d')
  start_date = three_years_ago.strftime('%Y-%m-%d')

  StockData_in_news_for_cal = yf.download(ticker_symbol, start = start_date_for_cal, end = end_date)
  StockData_in_news = yf.download(ticker_symbol, start = start_date, end = end_date)

  return(StockData_in_news_for_cal, StockData_in_news)

In [ ]:
# 2달 이동평균 계산 함수
def calculate_2M_MA_by_date(data, end_date):
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=2)
    mask = (data.index >= start_date) & (data.index < end_date)
    return data.loc[mask, 'Close'].mean()

In [ ]:
# 2달 이동평균값 계산 후 저장
def twoM_MA_Save(StockData_in_news_for_cal, StockData_in_news, today):
  today = today.date()
  two_month_MA_list = []
  current_date = today - pd.DateOffset(years=3)   # start_date = 3년전
  while current_date <= today:
    if current_date in StockData_in_news.index:
            two_month_MA = calculate_2M_MA_by_date(StockData_in_news_for_cal, current_date)
            two_month_MA_list.append(two_month_MA)
    current_date += pd.DateOffset(days=1)

  # 계산된 2달 이동평균값을 데이터프레임에 추가
  StockData_in_news['2M_MA'] = two_month_MA_list

In [ ]:
# 산업 타기업 평균 주가 리스트
def other_companies_stock(target_company, StockData_in_news):
  # 원하는 기업이 속한 업종 찾기
  for industry in business_dict.keys():
    if target_company in business_dict[industry]:
      target_industry = industry
      break

  # 해당 업종의 타 기업들의 주가 데이터 가져오기
  other_companies = [company for company in business_dict[target_industry] if company != target_company]

  # 각 기업의 주가 데이터를 평균내어 산업 평균 주가 계산
  industry_avg_price = pd.DataFrame()
  for company in other_companies:
    code = company_code[company]
    _, stock_data = load_stock(code)
    stock_data = stock_data['Close']
    industry_avg_price[company] = stock_data

  StockData_in_news['Industry_Avg'] = industry_avg_price.mean(axis=1)

  return(StockData_in_news)

In [ ]:
# 그래프 그리는 함수(3년)
def save_the_graph(StockData_in_news_for_cal, StockData_in_news, company):
  plt.figure(figsize=(18,6))

  ### 1번째 그래프
  plt.subplot(1,2,1)
  # 주가 그래프 그리는 함수(3년)
  plt.plot(StockData_in_news.index, StockData_in_news.Close, c='deeppink', label = f'{company}')
  # 2달 이동평균선을 그리는 함수
  today = datetime.today()   ## 오늘 날짜
  twoM_MA_Save(StockData_in_news_for_cal, StockData_in_news, today)
  plt.plot(StockData_in_news.index, StockData_in_news['2M_MA'], c='dodgerblue', label=f'{company}의 2달 이동평균')
  plt.legend(fontsize = 15)
  plt.title(f'{company} 주가 & 2달 이동평균선', fontsize=20)
  plt.xlabel('Date', fontsize=17)  # x축 레이블 크기 조절
  plt.ylabel('Price', fontsize=17)  # y축 레이블 크기 조절

  ### 2번째 그래프
  plt.subplot(1,2,2)
  # 퍼센트 변화 계산
  other_companies_stock(company, StockData_in_news)
  pct_change = StockData_in_news['Close'].pct_change().cumsum()
  industry_avg_pct_change = StockData_in_news['Industry_Avg'].pct_change().cumsum()
  # 주가 그래프 그리는 함수(3년 퍼센트 변화)
  plt.plot(StockData_in_news.index, pct_change, c='deeppink', label=f'{company} 퍼센트 변화')
  # 타기업 주가평균 그래프 그리는 함수(3년 퍼센트 변화)
  plt.plot(StockData_in_news.index, industry_avg_pct_change, c='yellowgreen', label=f'KOSPI 산업평균 퍼센트 변화')
  plt.legend(fontsize = 15)
  plt.title(f'{company} 주가 & KOSPI 산업평균 퍼센트 변화', fontsize=20)
  plt.xlabel('Date', fontsize=17)
  plt.ylabel('Percent Change', fontsize=17)

  plt.tight_layout()  # 그래프 간격 조정
  plt.savefig(path_graph + f'{company}.jpg')

In [ ]:
def analyze_graph(stock_data, company):
    analysis_result = []

    # 시작하는 말
    analysis_result.append(f"지난 3년 동안 {company}의 주가 추세를 분석한 결과,")
    # 1. 3년동안의 간단한 추세 분석
    # 시작점과 끝점의 주가 비교
    start_price = stock_data['Close'].iloc[0]
    end_price = stock_data['Close'].iloc[-1]
    # 이동평균선의 기울기 확인 (예: 2개월 이동평균선)
    ma_slope = stock_data['2M_MA'].iloc[-1] - stock_data['2M_MA'].iloc[0]

    if start_price < end_price and ma_slope > 0:
        analysis_result.append("3년동안의 주가는 전반적으로 상승하는 경향을 확인할 수 있습니다.")
    elif start_price > end_price and ma_slope < 0:
        analysis_result.append("3년동안의 주가는 전반적으로 하락하는 경향을 확인할 수 있습니다.")
    elif start_price < end_price and ma_slope < 0:
        analysis_result.append("3년동안의 주가는 전반적으로 상승하다가 하락하는 경향을 확인할 수 있습니다.")
    elif start_price > end_price and ma_slope > 0:
        analysis_result.append("3년동안의 주가는 전반적으로 하락하다가 상승하는 경향을 확인할 수 있습니다.")
    else:
        analysis_result.append("3년동안의 주가는 전반적으로 변동하는 경향을 확인할 수 있습니다.")

    # 잇는 말
    analysis_result.append(f"최근의 데이터를 살펴보면,")
    # 2. 최근 추세 확인
    if stock_data['Close'].iloc[-1] > stock_data['2M_MA'].iloc[-1]:
        analysis_result.append("현재 주가는 2개월 이동평균선 위에 있음을 알 수 있습니다. 이는 단기적인 상승 추세를 나타내며,")
    else:
        analysis_result.append("현재 주가는 2개월 이동평균선 아래에 있음을 알 수 있습니다. 이는 단기적인 하락 추세를 나타내며,")

    # 3. 이동평균선의 기울기 확인
    if stock_data['2M_MA'].iloc[-1] > stock_data['2M_MA'].iloc[-5]:
        analysis_result.append("특히 최근 2개월 이동평균선의 기울기가 긍정적인 것을 고려하면, 앞으로의 주가 상승 가능성을 시사하는 신호로 해석될 수 있습니다.")
    else:
        analysis_result.append("특히 최근 2개월 이동평균선의 기울기가 부정적입 것을 고려하면, 앞으로의 주가 하락 가능성을 시사하는 신호로 해석될 수 있습니다.")

    # 4. 타기업
    # 주가의 시작점과 마지막 점의 퍼센트 변화 계산
    company_pct_change = (stock_data['Close'].iloc[-1] - stock_data['Close'].iloc[0]) / stock_data['Close'].iloc[0] * 100
    industry_avg_pct_change = (stock_data['Industry_Avg'].iloc[-1] - stock_data['Industry_Avg'].iloc[0]) / stock_data['Industry_Avg'].iloc[0] * 100
    # 최근 트렌드 파악
    recent_trend = "상승" if stock_data['Close'].iloc[-1] > stock_data['Close'].iloc[0] else "하락"
    # 분석 결과를 바탕으로 글 작성
    analysis_result.append(f"\n지난 3년 간 {company}의 주가는 약 {company_pct_change:.2f}% {recent_trend}률을 기록하였습니다.")
    analysis_result.append(f"한편, 동일 기간 동안 KOSPI 산업 평균 주가는 약 {industry_avg_pct_change:.2f}%의 변화를 보였습니다.")
    if company_pct_change > industry_avg_pct_change:
        analysis_result.append(f"이러한 데이터를 통해 {company}의 주가 성장률이 산업 평균보다 높은 것을 확인할 수 있습니다.")
    else:
        analysis_result.append(f"이러한 데이터를 통해 {company}의 주가 성장률이 산업 평균보다 낮은 것을 확인할 수 있습니다.")

    return " ".join(analysis_result)

In [ ]:
# 각 뉴스별 그래프 저장 & report comment 저장
analysis_results = []
for i in range(len(final_code_list)):
  ex_for_cal, ex = load_stock(final_code_list[i])
  today = datetime.today()
  twoM_MA_Save(ex_for_cal, ex, today)
  company = final_company_list[i]
  save_the_graph(ex_for_cal, ex, company)
  analysis_results.append(analyze_graph(ex, company))

In [ ]:
# 감정분석 글도 추가하기
for i in range(len(analysis_results)) :
  if final['dominant_sentiment'][i] == 0:
    analysis_results[i] += "\n감정분석의 경우, 어떠한 방향으로도 치우치지 않는, 정보 제공을 목적으로 하는 기사이기에 'neutral'으로 분류되었습니다."
  elif final['dominant_sentiment'][i] == 1:
    analysis_results[i] += "\n감정분석의 경우, 금융 시장의 상승 또는 좋은 경제 지표에 대한 기사이기에 'positive'으로 분류되었습니다."
  elif final['dominant_sentiment'][i] == 2:
    analysis_results[i] += "\n감정분석의 경우 금융 시장의 하락 또는 부정적인 경제 지표에 대한 기사이기에 'negative'으로 분류되었습니다."

In [ ]:
final['analysis'] = analysis_results

In [ ]:
final['analysis'][0]

"지난 3년 동안 GS리테일의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 하락하는 경향을 확인할 수 있습니다. 최근의 데이터를 살펴보면, 현재 주가는 2개월 이동평균선 아래에 있음을 알 수 있습니다. 이는 단기적인 하락 추세를 나타내며, 특히 최근 2개월 이동평균선의 기울기가 긍정적인 것을 고려하면, 앞으로의 주가 상승 가능성을 시사하는 신호로 해석될 수 있습니다. \n지난 3년 간 GS리테일의 주가는 약 -37.76% 하락률을 기록하였습니다. 한편, 동일 기간 동안 KOSPI 산업 평균 주가는 약 -15.95%의 변화를 보였습니다. 이러한 데이터를 통해 GS리테일의 주가 성장률이 산업 평균보다 낮은 것을 확인할 수 있습니다.\n감정분석의 경우, 어떠한 방향으로도 치우치지 않는, 정보 제공을 목적으로 하는 기사이기에 'neutral'으로 분류되었습니다."

# Final Dataframe

In [ ]:
# 컬럼들을 선택하여 새로운 DataFrame으로 합치기
new_df = pd.concat([
                    art_df['summary'],
                    final['dominant_sentiment'],
                    final_company_df['Company Name'],
                    df_links['url'],
                    final['analysis'],
                    final_news_list['title']
                   ],
                    axis=1)

In [ ]:
new_df

,summary,dominant_sentiment,Company Name,url,analysis,title
0,11일 GS리테일은 카카오 커머스 CIC와 전략적 제휴를 맺어 주류 스마트 오더 플...,0,GS리테일,https://n.news.naver.com/mnews/article/001/001...,"지난 3년 동안 GS리테일의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로...",카카오 손잡은 GS리테일…선물하기로 주류 판매 서비스 확대
1,"LG에너지솔루션은 11일 잠정실적 공시를 통해 올해 3분기 매출액 8조2235억원,...",1,LG에너지솔루션,https://n.news.naver.com/mnews/article/031/000...,"지난 3년 동안 LG에너지솔루션의 주가 추세를 분석한 결과, 3년동안의 주가는 전반...","LG에너지솔루션, 3분기 영업이익 7312억원…""사상 최대 실적"""
2,오비맥주가 11일부터 주요 맥주 제품의 공장 출고가를 평균 6.9% 인상한다고 하는...,1,하이트진로,https://n.news.naver.com/mnews/article/001/001...,"지난 3년 동안 하이트진로의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로...","오늘부터 맥주 오른다…오비맥주, 출고가 6.9% 인상"
3,사세세습을 두고 국내 5개 완성차업체 중 현재 유일하게 임금 및 단체협상을 마무리하...,0,현대차,https://n.news.naver.com/mnews/article/018/000...,"지난 3년 동안 현대차의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상...","현대차도 없앤 ‘고용세습’ 고수…기아 노조, 12일부터 파업"
4,자동차 정비업체의 수리비를 허위·과장 청구하는 보험사기로 처벌받는 사례가 지속적으로...,0,SBS,https://n.news.naver.com/mnews/article/374/000...,"지난 3년 동안 SBS의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상...",車 무상수리 권유엔 의심부터…보험사기 연루 '주의보'
5,KG모빌리티는 전남 담양군에 위치한 ‘도래수 마을’을 국내 최초 친환경 전기차 마을...,1,KG모빌리티,https://n.news.naver.com/mnews/article/119/000...,"지난 3년 동안 KG모빌리티의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으...","'토레스 EVX' 1호차는 '도래수' 마을로… KG모빌리티, 교통약자 돕는다"
6,LG전자가 올레드 화질과 인기 게임 ‘리그 오브 레전드’ 디자인을 갖춘 ‘LG 울트...,0,LG,https://n.news.naver.com/mnews/article/016/000...,"지난 3년 동안 LG의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상승...",LG전자 ‘LG 울트라기어 게이밍모니터 LoL(롤) 에디션’ 출시
7,김1일 국회 정무위원회 금융위 국정감사에 출석한 김주현 위원장은 고객 몰래 1000...,2,DGB금융지주,https://n.news.naver.com/mnews/article/421/000...,"지난 3년 동안 DGB금융지주의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적...","김주현 금융위원장 ""대구은행 시중은행 전환, 대주주 적격성 등 살필 것"""
8,11일 업계에 따르면 민주노총 금속노조 기아차지부는 이날 노조 소식지를 통해 임단협...,2,DB,https://n.news.naver.com/mnews/article/003/001...,"지난 3년 동안 DB의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상승...","'파업 선언' 기아 노조…""11개월 대기 소비자는 외면"""
9,,1,대신증권,https://n.news.naver.com/mnews/article/016/000...,"지난 3년 동안 대신증권의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 ...",“삼성 D램이 살아난다”…‘10조 적자’ 딛고 완벽 부활 신호탄 [비즈360]


# + 로고 이미지 jpg로 변형, 로고 이미지 리사이징 (필요시)

In [ ]:
'''
image_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/'
all_files = os.listdir(image_path)
for i in range(len(all_files)):
  if all_files[i].endswith('.webp'):
    print(all_files[i])
    '''

"\nimage_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/'\nall_files = os.listdir(image_path)\nfor i in range(len(all_files)):\n  if all_files[i].endswith('.webp'):\n    print(all_files[i])\n    "

In [ ]:
'''
# 로고 사진 jpg 파일로 변환
def convert_images_to_jpg(directory):
    # 지정된 디렉토리 내의 모든 파일을 순회
    for filename in os.listdir(directory):
        # 파일 확장자 확인
        if filename.endswith('.gif') or filename.endswith('.png') or filename.endswith('.jpeg'):
            # 이미지 파일 열기
            with Image.open(os.path.join(directory, filename)) as img:
                # 파일명에서 확장자 제거
                base_filename = os.path.splitext(filename)[0]
                # jpg로 저장
                img.convert('RGB').save(os.path.join(directory, base_filename + '.jpg'))
                # 원본 파일 삭제 (선택 사항)
                os.remove(os.path.join(directory, filename))
                print(f"Converted {filename} to {base_filename}.jpg")

# 사용
directory_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/'  # 이미지 파일이 있는 디렉토리 경로
convert_images_to_jpg(directory_path)
'''

'\n# 로고 사진 jpg 파일로 변환\ndef convert_images_to_jpg(directory):\n    # 지정된 디렉토리 내의 모든 파일을 순회\n    for filename in os.listdir(directory):\n        # 파일 확장자 확인\n        if filename.endswith(\'.gif\') or filename.endswith(\'.png\') or filename.endswith(\'.jpeg\'):\n            # 이미지 파일 열기\n            with Image.open(os.path.join(directory, filename)) as img:\n                # 파일명에서 확장자 제거\n                base_filename = os.path.splitext(filename)[0]\n                # jpg로 저장\n                img.convert(\'RGB\').save(os.path.join(directory, base_filename + \'.jpg\'))\n                # 원본 파일 삭제 (선택 사항)\n                os.remove(os.path.join(directory, filename))\n                print(f"Converted {filename} to {base_filename}.jpg")\n\n# 사용\ndirectory_path = \'/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/\'  # 이미지 파일이 있는 디렉토리 경로\nconvert_images_to_jpg(directory_path)\n'

In [ ]:
'''
# 로고 이미지 리사이징
def resize_and_save_images_modified(image_path, save_path):
    # 해당 경로에서 모든 파일 및 디렉토리의 이름을 가져옵니다.
    all_files = os.listdir(image_path)
    for file_path in all_files:
        with Image.open(image_path+file_path) as img:
            # Calculate the aspect ratio while keeping one dimension fixed
            aspect_ratio = img.width / img.height
            new_width = int(93 * aspect_ratio)
            new_height = 93
            if new_width > 277:
                new_width = 277
                new_height = int(277 / aspect_ratio)

            img_resized = img.resize((new_width, new_height))
            file_name = os.path.basename(file_path)
            save_file_path = os.path.join(save_path, file_name)
            img_resized.save(save_file_path)

save_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/resized_logos/'
image_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/'
resize_and_save_images_modified(image_path, save_path)
'''

"\n# 로고 이미지 리사이징\ndef resize_and_save_images_modified(image_path, save_path):\n    # 해당 경로에서 모든 파일 및 디렉토리의 이름을 가져옵니다.\n    all_files = os.listdir(image_path)\n    for file_path in all_files:\n        with Image.open(image_path+file_path) as img:\n            # Calculate the aspect ratio while keeping one dimension fixed\n            aspect_ratio = img.width / img.height\n            new_width = int(93 * aspect_ratio)\n            new_height = 93\n            if new_width > 277:\n                new_width = 277\n                new_height = int(277 / aspect_ratio)\n\n            img_resized = img.resize((new_width, new_height))\n            file_name = os.path.basename(file_path)\n            save_file_path = os.path.join(save_path, file_name)\n            img_resized.save(save_file_path)\n\nsave_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/resized_logos/'\nimage_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/'\nresize_and_save_images_modified(image_path, save_pa

In [ ]:
'''
# 기업 한글이름 영어로 변환해서 match시킬 딕셔너리 만들기
# 변환기 생성
transliter = Transliter(academic)

match_company_list_inE = {}
for x in company_list:
  new_x = re.sub(r'\s+|[^a-zA-Z]', '', transliter.translit(x))
  match_company_list_inE[x] = new_x

print(match_company_list_inE)
'''

"\n# 기업 한글이름 영어로 변환해서 match시킬 딕셔너리 만들기\n# 변환기 생성\ntransliter = Transliter(academic)\n\nmatch_company_list_inE = {}\nfor x in company_list:\n  new_x = re.sub(r'\\s+|[^a-zA-Z]', '', transliter.translit(x))\n  match_company_list_inE[x] = new_x\n\nprint(match_company_list_inE)\n"

In [ ]:
'''
# 로고 이미지 이름 영어로 바꾸기
# 폴더 경로
folder_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/'

# 폴더 내의 모든 파일을 나열합니다.
files = os.listdir(folder_path)

# 영어+숫자 문자 패턴
pattern_eng_num = re.compile('^[a-zA-Z0-9]+$')

# 파일 이름 변경
for filename in files:
    # 파일 확장자 확인
    extension = os.path.splitext(filename)[1]

    # 이미 영어+숫자로 바뀐 파일 이름인 경우 건너뛰기
    if pattern_eng_num.search(os.path.splitext(filename)[0].replace("_logo", "")):
        continue
    else:
      korname = unicodedata.normalize('NFC', os.path.splitext(filename)[0].replace("_logo", ""))
      # 새 파일 이름 (예: image1.jpg, image2.jpg, ...)
      new_filename = f"{match_company_list_inE[korname]}_logo{extension}"
      # 파일 이름 변경
      os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))
      '''

'\n# 로고 이미지 이름 영어로 바꾸기\n# 폴더 경로\nfolder_path = \'/content/drive/My Drive/SNU_JYS/NLP/report_try/logos/\'\n\n# 폴더 내의 모든 파일을 나열합니다.\nfiles = os.listdir(folder_path)\n\n# 영어+숫자 문자 패턴\npattern_eng_num = re.compile(\'^[a-zA-Z0-9]+$\')\n\n# 파일 이름 변경\nfor filename in files:\n    # 파일 확장자 확인\n    extension = os.path.splitext(filename)[1]\n\n    # 이미 영어+숫자로 바뀐 파일 이름인 경우 건너뛰기\n    if pattern_eng_num.search(os.path.splitext(filename)[0].replace("_logo", "")):\n        continue\n    else:\n      korname = unicodedata.normalize(\'NFC\', os.path.splitext(filename)[0].replace("_logo", ""))\n      # 새 파일 이름 (예: image1.jpg, image2.jpg, ...)\n      new_filename = f"{match_company_list_inE[korname]}_logo{extension}"\n      # 파일 이름 변경\n      os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))\n      '

In [ ]:
'''
# 그래프 이미지 이름 영어로 바꾸기
# 폴더 경로
folder_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/graphs/'

# 폴더 내의 모든 파일을 나열합니다.
files = os.listdir(folder_path)

# 영어+숫자 문자 패턴
pattern_eng_num = re.compile('^[a-zA-Z0-9]+$')

# 파일 이름 변경
for filename in files:
    # 파일 확장자 확인
    extension = os.path.splitext(filename)[1]

    # 이미 영어+숫자로 바뀐 파일 이름인 경우 건너뛰기
    if pattern_eng_num.search(os.path.splitext(filename)[0]):
        continue
    else:
      korname = unicodedata.normalize('NFC', os.path.splitext(filename)[0])
      # 새 파일 이름 (예: image1.jpg, image2.jpg, ...)
      new_filename = f"{match_company_list_inE[korname]}{extension}"
      # 파일 이름 변경
      os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))
      '''

'\n# 그래프 이미지 이름 영어로 바꾸기\n# 폴더 경로\nfolder_path = \'/content/drive/My Drive/SNU_JYS/NLP/report_try/graphs/\'\n\n# 폴더 내의 모든 파일을 나열합니다.\nfiles = os.listdir(folder_path)\n\n# 영어+숫자 문자 패턴\npattern_eng_num = re.compile(\'^[a-zA-Z0-9]+$\')\n\n# 파일 이름 변경\nfor filename in files:\n    # 파일 확장자 확인\n    extension = os.path.splitext(filename)[1]\n\n    # 이미 영어+숫자로 바뀐 파일 이름인 경우 건너뛰기\n    if pattern_eng_num.search(os.path.splitext(filename)[0]):\n        continue\n    else:\n      korname = unicodedata.normalize(\'NFC\', os.path.splitext(filename)[0])\n      # 새 파일 이름 (예: image1.jpg, image2.jpg, ...)\n      new_filename = f"{match_company_list_inE[korname]}{extension}"\n      # 파일 이름 변경\n      os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))\n      '

# 보고서 작성 자동화

In [ ]:
# 기업 한글이름 영어로 변환해서 match시킬 딕셔너리 만들기
# 변환기 생성
transliter = Transliter(academic)

match_company_list_inE = {}
for x in company_list:
  new_x = re.sub(r'\s+|[^a-zA-Z]', '', transliter.translit(x))
  match_company_list_inE[x] = new_x

print(match_company_list_inE)

{'AJ네트웍스': 'AJneteuwogseu', 'AK홀딩스': 'AKholdingseu', 'BGF': 'BGF', 'BGF리테일': 'BGFliteil', 'BNK금융지주': 'BNKgeumyungjiju', 'BYC': 'BYC', 'BYC우': 'BYCu', 'CJ': 'CJ', 'CJ CGV': 'CJCGV', 'CJ4우(전환)': 'CJujeonhwan', 'CJ대한통운': 'CJdaehantongun', 'CJ씨푸드': 'CJssipudeu', 'CJ씨푸드1우': 'CJssipudeuu', 'CJ우': 'CJu', 'CJ제일제당': 'CJjeiljedang', 'CJ제일제당 우': 'CJjeiljedangu', 'CR홀딩스': 'CRholdingseu', 'CS홀딩스': 'CSholdingseu', 'DB': 'DB', 'DB금융투자': 'DBgeumyungtuja', 'DB손해보험': 'DBsonhaeboheom', 'DB하이텍': 'DBhaiteg', 'DGB금융지주': 'DGBgeumyungjiju', 'DI동일': 'DIdongil', 'DL': 'DL', 'DL건설': 'DLgeonseol', 'DL우': 'DLu', 'DL이앤씨': 'DLiaenssi', 'DL이앤씨2우(전환)': 'DLiaenssiujeonhwan', 'DL이앤씨우': 'DLiaenssiu', 'DN오토모티브': 'DNotomotibeu', 'DRB동일': 'DRBdongil', 'DSR': 'DSR', 'DSR제강': 'DSRjegang', 'E1': 'E', 'ESR켄달스퀘어리츠': 'ESRkendalseukweeolicheu', 'F&F': 'FF', 'F&F홀딩스': 'FFholdingseu', 'GKL': 'GKL', 'GS': 'GS', 'GS건설': 'GSgeonseol', 'GS글로벌': 'GSgeullobeol', 'GS리테일': 'GSliteil', 'GS우': 'GSu', 'HDC': 'HDC', 'HDC랩스': 'HDClaebseu', 'HDC현

In [ ]:
# 그래프 이미지 이름 영어로 바꾸기
# 폴더 경로
folder_path = '/content/drive/My Drive/SNU_JYS/NLP/report_try/graphs/'

# 폴더 내의 모든 파일을 나열합니다.
files = os.listdir(folder_path)

# 영어+숫자 문자 패턴
pattern_eng_num = re.compile('^[a-zA-Z0-9]+$')

# 파일 이름 변경
for filename in files:
    # 파일 확장자 확인
    extension = os.path.splitext(filename)[1]

    # 이미 영어+숫자로 바뀐 파일 이름인 경우 건너뛰기
    if pattern_eng_num.search(os.path.splitext(filename)[0]):
        continue
    else:
      korname = unicodedata.normalize('NFC', os.path.splitext(filename)[0])
      # 새 파일 이름 (예: image1.jpg, image2.jpg, ...)
      new_filename = f"{match_company_list_inE[korname]}{extension}"
      # 파일 이름 변경
      os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))

In [ ]:
new_df

,summary,dominant_sentiment,Company Name,url,analysis,title
0,11일 GS리테일은 카카오 커머스 CIC와 전략적 제휴를 맺어 주류 스마트 오더 플...,0,GS리테일,https://n.news.naver.com/mnews/article/001/001...,"지난 3년 동안 GS리테일의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로...",카카오 손잡은 GS리테일…선물하기로 주류 판매 서비스 확대
1,"LG에너지솔루션은 11일 잠정실적 공시를 통해 올해 3분기 매출액 8조2235억원,...",1,LG에너지솔루션,https://n.news.naver.com/mnews/article/031/000...,"지난 3년 동안 LG에너지솔루션의 주가 추세를 분석한 결과, 3년동안의 주가는 전반...","LG에너지솔루션, 3분기 영업이익 7312억원…""사상 최대 실적"""
2,오비맥주가 11일부터 주요 맥주 제품의 공장 출고가를 평균 6.9% 인상한다고 하는...,1,하이트진로,https://n.news.naver.com/mnews/article/001/001...,"지난 3년 동안 하이트진로의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로...","오늘부터 맥주 오른다…오비맥주, 출고가 6.9% 인상"
3,사세세습을 두고 국내 5개 완성차업체 중 현재 유일하게 임금 및 단체협상을 마무리하...,0,현대차,https://n.news.naver.com/mnews/article/018/000...,"지난 3년 동안 현대차의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상...","현대차도 없앤 ‘고용세습’ 고수…기아 노조, 12일부터 파업"
4,자동차 정비업체의 수리비를 허위·과장 청구하는 보험사기로 처벌받는 사례가 지속적으로...,0,SBS,https://n.news.naver.com/mnews/article/374/000...,"지난 3년 동안 SBS의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상...",車 무상수리 권유엔 의심부터…보험사기 연루 '주의보'
5,KG모빌리티는 전남 담양군에 위치한 ‘도래수 마을’을 국내 최초 친환경 전기차 마을...,1,KG모빌리티,https://n.news.naver.com/mnews/article/119/000...,"지난 3년 동안 KG모빌리티의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으...","'토레스 EVX' 1호차는 '도래수' 마을로… KG모빌리티, 교통약자 돕는다"
6,LG전자가 올레드 화질과 인기 게임 ‘리그 오브 레전드’ 디자인을 갖춘 ‘LG 울트...,0,LG,https://n.news.naver.com/mnews/article/016/000...,"지난 3년 동안 LG의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상승...",LG전자 ‘LG 울트라기어 게이밍모니터 LoL(롤) 에디션’ 출시
7,김1일 국회 정무위원회 금융위 국정감사에 출석한 김주현 위원장은 고객 몰래 1000...,2,DGB금융지주,https://n.news.naver.com/mnews/article/421/000...,"지난 3년 동안 DGB금융지주의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적...","김주현 금융위원장 ""대구은행 시중은행 전환, 대주주 적격성 등 살필 것"""
8,11일 업계에 따르면 민주노총 금속노조 기아차지부는 이날 노조 소식지를 통해 임단협...,2,DB,https://n.news.naver.com/mnews/article/003/001...,"지난 3년 동안 DB의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 상승...","'파업 선언' 기아 노조…""11개월 대기 소비자는 외면"""
9,,1,대신증권,https://n.news.naver.com/mnews/article/016/000...,"지난 3년 동안 대신증권의 주가 추세를 분석한 결과, 3년동안의 주가는 전반적으로 ...",“삼성 D램이 살아난다”…‘10조 적자’ 딛고 완벽 부활 신호탄 [비즈360]


In [ ]:
width, height = letter
print(width, height)

In [ ]:
print(match_company_list_inE['NH투자증권'])

In [ ]:
# PDF 생성
today = datetime.today()
c = canvas.Canvas(f"{today.year}년 {today.month}월 {today.day}일 Daily News Investment Reports.pdf", pagesize=letter)
width, height = letter

# 제목 페이지 추가
c.setFont("NotoSansKR", 30)
c.drawCentredString(width / 2, height / 2 + 120, "Daily News Investment Reports")
c.setFont("NotoSansKR", 18)
c.drawCentredString(width / 2, height / 2, f"{today.year}년 {today.month}월 {today.day}일")
c.setFont("NotoSansKR", 15)
c.drawCentredString(width / 2, height / 2 - 50, "윤주희, 박수민, 강예지")
c.showPage()  # 제목 페이지 완료 후 다음 페이지로 이동

for index, row in new_df.iterrows():
  #뉴스기사제목
  c.setFont("NotoSansKR", 20)
  c.drawCentredString(width / 2, height / 2 + 300, f"{row['title']}")
  # 밑줄 추가
  text_width = stringWidth(row['title'], "NotoSansKR", 24)
  underline_position = height / 2 + 300 - 5  # 밑줄의 위치 (5는 밑줄과 문자열 사이의 간격)
  c.line(width / 2 - text_width / 2, underline_position, width / 2 + text_width / 2, underline_position)

  ### 요약
  style = ParagraphStyle(
        name="NotoSansKR",
        fontName='NotoSansKR',  # 한글 폰트 지정
        fontSize=13,
        leading=15
    )
  # Paragraph 객체를 생성해서 텍스트와 스타일을 지정
  long_text = row['summary']
  paragraph = Paragraph('요약 : <br/>'+long_text, style)
  # drawOn 메소드를 사용해서 Paragraph를 캔버스에 그림
  paragraph.wrapOn(c, width - 2 * 72, height)
  paragraph.drawOn(c, 72, height / 2 + 200)

  ### 감정분류
  c.setFont("NotoSansKR", 13)
  if row['dominant_sentiment'] == 0 :
    sense = 'Neutral'
    c.drawCentredString(137, height / 2 + 160, f"감정분석 결과 : {sense}")
  elif row['dominant_sentiment'] == 1 :
    sense = 'Negative'
    c.drawCentredString(142, height / 2 + 160, f"감정분석 결과 : {sense}")
  elif row['dominant_sentiment'] == 2 :
    sense = 'Positive'
    c.drawCentredString(137, height / 2 + 160, f"감정분석 결과 : {sense}")

  ### 종목명, 종목코드
  c.setFont("NotoSansKR", 18)
  c.drawCentredString(width / 3, height / 2 + 80, f"종목명 : {row['Company Name']}")
  c.setFont("NotoSansKR", 16)
  code = company_code[row['Company Name']]
  c.drawCentredString(width / 3, height / 2 + 60, f"종목코드 : {code}")

  # 로고 이미지 추가
  logo_name = match_company_list_inE[row['Company Name']]
  logo_path = f'/content/drive/MyDrive/SNU_JYS/NLP/report_try/logos/{logo_name}_logo.jpg'
  c.drawImage(logo_path, width / 2 + 50, height / 2 + 10, width = 160, height=120, mask='auto')

  # 테두리 그리기
  left = 612 / 2 - 200  # 테두리의 왼쪽 좌표
  right = 612 / 2 + 200  # 테두리의 오른쪽 좌표
  top = 150  # 테두리의 위쪽 좌표
  bottom = 350  # 테두리의 아래쪽 좌표
  c.rect(306-235, 400, 470, 135, stroke=1, fill=0)

  # 그래프 이미지 추가
  graph_name = match_company_list_inE[row['Company Name']]
  graph_path = f'/content/drive/MyDrive/SNU_JYS/NLP/report_try/graphs/{graph_name}.jpg'
  c.drawImage(graph_path, 56, height / 2 - 195, width = 500, height = 180, mask='auto')

  ### comment
  # ParagraphStyle을 생성해서 문단의 스타일을 지정
  style = ParagraphStyle(
        name="NotoSansKR",
        fontName='NotoSansKR',  # 한글 폰트 지정
        fontSize=10,
        leading=17
    )
  # Paragraph 객체를 생성해서 텍스트와 스타일을 지정
  long_text2 = row['analysis']
  paragraph = Paragraph(long_text2, style)
  # drawOn 메소드를 사용해서 Paragraph를 캔버스에 그림
  paragraph.wrapOn(c, width - 2 * 72, height)
  paragraph.drawOn(c, 72, height / 2 - 330)

  # url
  c.setFont("NotoSansKR", 10)
  c.drawCentredString(400, height / 2 - 350, f"{row['url']}")

  # 다음 페이지로 넘어감
  c.showPage()

# PDF 저장
c.save()